In [1]:
import pandas as pd
import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

%load_ext blackcellmagic

Loading BokehJS ...

Exercise 3.1: Automating scatter plots

Write a function that takes as input a tidy data frame and generates a scatter plot based on two columns of the data frame and colors the glyphs according to a third column that contains categorical variables. The minimal (you can add other kwargs if you want) call signature should be

scatter(data, cat, x, y)
You will of course test out your function while writing it, and the next problems give you lots of opportunities to use it.

In [2]:
# I'm honestly not sure how to test a plotting function, so this is vague for now
def test_scatter_makes_plot():
    """Checks that scatter() makes a bokeh Figure."""
    assert type(scatter(data, cat, x, y)) == bokeh.plotting.figure.Figure

In [3]:
# I've decided to make the cat arg optional, so it has to go after the required args
def scatter(data, x, y, cat=None):
    # later I want to add other kwargs for width, height, etc.
    """Produces a bokeh scatter plot from a tidy data frame.
    
    Args:
        data: the identity of the data frame from which to plot
        x: the name of the column in the data frame to make the x variable, given as a string
        y: the name of the column in the data frame to make the y variable, given as a string
        cat: the name of the column in the data frame to color-code the data, given as a string.
            Data will be treated as categorical. If none given, all data will be the same color.
            
    Returns:
        A bokeh scatter plot."""

    # make the canvas
    p = bokeh.plotting.figure(
        # taking optional args
        x_axis_label=x,
        y_axis_label=y,
    )

    if cat == None:
        p.circle(source=data, x=x, y=y)

        return p

    cat_vars = data[cat].unique()

    # initialize set of colors here - would like to make bigger eventually
    # this is ugly but hey
    colors = ["red", "orange", "green", "blue"]
    i = 0

    for var in cat_vars:
        p.circle(
            source=df.loc[df[cat] == var, :],
            x=x,
            y=y,
            color=colors[i],
            legend_label=var,
        )

        i += 1
        # repeat colors if too many cat variables
        if i >= len(colors):
            i = 0

    return p

In [4]:
df = pd.read_csv('data/frog_tongue_adhesion.csv', comment='#')

#make note of how to make a dataframe from scratch. Could be useful for testing
df_frog = pd.DataFrame(data={'ID': ['I', 'II', 'III', 'IV'],
                             'age': ['adult', 'adult', 'juvenile', 'juvenile'],
                             'SVL (mm)': [63, 70, 28, 31],
                             'weight (g)': [63.1, 72.7, 12.7, 12.7],
                             'species': ['cross', 'cross', 'cranwelli', 'cranwelli']})

df.head()

,date,ID,trial number,impact force (mN),impact time (ms),impact force / body weight,adhesive force (mN),time frog pulls on target (ms),adhesive force / body weight,adhesive impulse (N-s),total contact area (mm2),contact area without mucus (mm2),contact area with mucus / contact area without mucus,contact pressure (Pa),adhesive strength (Pa)
0,2013_02_26,I,3,1205,46,1.95,-785,884,1.27,-0.290,387,70,0.82,3117,-2030
1,2013_02_26,I,4,2527,44,4.08,-983,248,1.59,-0.181,101,94,0.07,24923,-9695
2,2013_03_01,I,1,1745,34,2.82,-850,211,1.37,-0.157,83,79,0.05,21020,-10239
3,2013_03_01,I,2,1556,41,2.51,-455,1025,0.74,-0.170,330,158,0.52,4718,-1381
4,2013_03_01,I,3,493,36,0.80,-974,499,1.57,-0.423,245,216,0.12,2012,-3975


In [5]:
plot = scatter(df, 'impact force (mN)', 'impact time (ms)')
bokeh.io.show(plot)

In [6]:
plot = scatter(df, "impact force (mN)", "impact time (ms)", cat="ID")
bokeh.io.show(plot)

In [7]:
# update test to reference real data
def test_scatter_makes_plot():
    """Checks that scatter() makes a bokeh Figure."""
    output_fig = scatter(df, "impact force (mN)", "impact time (ms)")
    assert str(type(output_fig)) == "<class 'bokeh.plotting.figure.Figure'>"


def test_scatter_all():
    test_scatter_makes_plot()
    print("Success!")

In [8]:
test_scatter_all()

Success!


In [9]:
#here's Justin's code:
def scatter(
    data=None,
    cat=None,
    x=None,
    y=None,
    p=None,
    palette=[
        "#4e79a7",
        "#f28e2b",
        "#e15759",
        "#76b7b2",
        "#59a14f",
        "#edc948",
        "#b07aa1",
        "#ff9da7",
        "#9c755f",
        "#bab0ac",
    ],
    show_legend=True,
    click_policy="hide",
    marker_kwargs={},
    **kwargs,
):
    """
    Parameters
    ----------
    df : Pandas DataFrame
        DataFrame containing tidy data for plotting.
    cat : hashable
        Name of column to use as categorical variable.
    x : hashable
        Name of column to use as x-axis.
    y : hashable
        Name of column to use as y-axis.
    p : bokeh.plotting.Figure instance, or None (default)
        If None, create a new figure. Otherwise, populate the existing
        figure `p`.
    palette : list of strings of hex colors, or single hex string
        If a list, color palette to use. If a single string representing
        a hex color, all glyphs are colored with that color. Default is
        the default color cycle employed by Vega-Lite.
    show_legend : bool, default False
        If True, show legend.
    tooltips : list of 2-tuples
        Specification for tooltips as per Bokeh specifications. For
        example, if we want `col1` and `col2` tooltips, we can use
        `tooltips=[('label 1': '@col1'), ('label 2': '@col2')]`. Ignored
        if `formal` is True.
    show_legend : bool, default False
        If True, show a legend.
    click_policy : str, default "hide"
        How to display points when their legend entry is clicked.
    marker_kwargs : dict
        kwargs to be passed to `p.circle()` when making the scatter plot.
    kwargs
        Any kwargs to be passed to `bokeh.plotting.figure()` when making 
        the plot.

    Returns
    -------
    output : bokeh.plotting.Figure instance
        Plot populated with jitter plot or box plot.
    """
    # Automatically name the axes
    if "x_axis_label" not in kwargs:
        kwargs["x_axis_label"] = x
    if "y_axis_label" not in kwargs:
        kwargs["y_axis_label"] = y

    # Instantiate figure
    if p is None:
        p = bokeh.plotting.figure(**kwargs)

    # Build plot (not using color factors) to enable click policies
    for i, (name, g) in enumerate(data.groupby(cat, sort=False)):
        marker_kwargs["color"] = palette[i % len(palette)]
        #Emily modified this line to avoid a deprecation warning
        marker_kwargs["legend_label"] = str(name)
        p.circle(source=g, x=x, y=y, **marker_kwargs)

    if show_legend:
        p.legend.click_policy = click_policy
    else:
        p.legend.visible = False

    return p

In [10]:
plot = scatter(df, x="impact force (mN)", y="impact time (ms)", cat="ID")
bokeh.io.show(plot)

Exercise 3.2: Adding data to a DataFrame

So, each frog has associated with it an age (adult or juvenile), snout-vent-length (SVL), body weight, and species (either cross or cranwelli). For a tidy DataFrame, we should have a column for each of these values. Your task is to load in the data, and then add these columns to the DataFrame.

Note: There are lots of ways to solve this problem. This is a good exercise in searching through the Pandas documentation and other online resources, such as Stack Overflow. Remember, much of your programming efforts are spent searching through documentation and the internet.

After you have added this information to the data frame, make a scatter plot of adhesive force versus impact force and color the points by whether the frog is a juvenile or adult. The function you wrote in Exercise 3.1 will be useful to do this.

In [11]:
df_frog

,ID,age,SVL (mm),weight (g),species
0,I,adult,63,63.1,cross
1,II,adult,70,72.7,cross
2,III,juvenile,28,12.7,cranwelli
3,IV,juvenile,31,12.7,cranwelli


In [12]:
#this goes badly
df_with_details = pd.concat([df, df_frog])

df_with_details

,date,ID,trial number,impact force (mN),impact time (ms),impact force / body weight,adhesive force (mN),time frog pulls on target (ms),adhesive force / body weight,adhesive impulse (N-s),total contact area (mm2),contact area without mucus (mm2),contact area with mucus / contact area without mucus,contact pressure (Pa),adhesive strength (Pa),age,SVL (mm),weight (g),species
0,2013_02_26,I,3.0,1205.0,46.0,1.95,-785.0,884.0,1.27,-0.290,387.0,70.0,0.82,3117.0,-2030.0,NaN,NaN,NaN,NaN
1,2013_02_26,I,4.0,2527.0,44.0,4.08,-983.0,248.0,1.59,-0.181,101.0,94.0,0.07,24923.0,-9695.0,NaN,NaN,NaN,NaN
2,2013_03_01,I,1.0,1745.0,34.0,2.82,-850.0,211.0,1.37,-0.157,83.0,79.0,0.05,21020.0,-10239.0,NaN,NaN,NaN,NaN
3,2013_03_01,I,2.0,1556.0,41.0,2.51,-455.0,1025.0,0.74,-0.170,330.0,158.0,0.52,4718.0,-1381.0,NaN,NaN,NaN,NaN
4,2013_03_01,I,3.0,493.0,36.0,0.80,-974.0,499.0,1.57,-0.423,245.0,216.0,0.12,2012.0,-3975.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,2013_06_21,IV,4.0,468.0,36.0,3.48,-218.0,1269.0,1.63,-0.122,129.0,68.0,0.61,3617.0,-1688.0,NaN,NaN,NaN,NaN
0,NaN,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,adult,63.0,63.1,cross
1,NaN,II,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,adult,70.0,72.7,cross
2,NaN,III,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,juvenile,28.0,12.7,cranwelli


In [13]:
#but I interestingly discovered that two rows can have the same index
df_with_details.loc[0,:]

,date,ID,trial number,impact force (mN),impact time (ms),impact force / body weight,adhesive force (mN),time frog pulls on target (ms),adhesive force / body weight,adhesive impulse (N-s),total contact area (mm2),contact area without mucus (mm2),contact area with mucus / contact area without mucus,contact pressure (Pa),adhesive strength (Pa),age,SVL (mm),weight (g),species
0,2013_02_26,I,3.0,1205.0,46.0,1.95,-785.0,884.0,1.27,-0.29,387.0,70.0,0.82,3117.0,-2030.0,NaN,NaN,NaN,NaN
0,NaN,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,adult,63.0,63.1,cross


In [14]:
#but this seems to work
df_with_details = pd.merge(df, df_frog)

df_with_details

,date,ID,trial number,impact force (mN),impact time (ms),impact force / body weight,adhesive force (mN),time frog pulls on target (ms),adhesive force / body weight,adhesive impulse (N-s),total contact area (mm2),contact area without mucus (mm2),contact area with mucus / contact area without mucus,contact pressure (Pa),adhesive strength (Pa),age,SVL (mm),weight (g),species
0,2013_02_26,I,3,1205,46,1.95,-785,884,1.27,-0.290,387,70,0.82,3117,-2030,adult,63,63.1,cross
1,2013_02_26,I,4,2527,44,4.08,-983,248,1.59,-0.181,101,94,0.07,24923,-9695,adult,63,63.1,cross
2,2013_03_01,I,1,1745,34,2.82,-850,211,1.37,-0.157,83,79,0.05,21020,-10239,adult,63,63.1,cross
3,2013_03_01,I,2,1556,41,2.51,-455,1025,0.74,-0.170,330,158,0.52,4718,-1381,adult,63,63.1,cross
4,2013_03_01,I,3,493,36,0.80,-974,499,1.57,-0.423,245,216,0.12,2012,-3975,adult,63,63.1,cross
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2013_06_18,IV,4,402,38,3.00,-302,986,2.25,-0.122,117,30,0.07,3446,-2591,juvenile,31,12.7,cranwelli
76,2013_06_21,IV,1,605,39,4.50,-216,1627,1.61,-0.139,123,20,1.00,4928,-1759,juvenile,31,12.7,cranwelli
77,2013_06_21,IV,2,711,76,5.30,-163,2021,1.21,-0.217,129,42,0.97,5498,-1257,juvenile,31,12.7,cranwelli
78,2013_06_21,IV,3,614,33,4.57,-367,1366,2.73,-0.198,128,108,0.46,4776,-2857,juvenile,31,12.7,cranwelli


After you have added this information to the data frame, make a scatter plot of adhesive force versus impact force and color the points by whether the frog is a juvenile or adult. The function you wrote in Exercise 3.1 will be useful to do this.

In [15]:
df_with_details.columns

Index(['date', 'ID', 'trial number', 'impact force (mN)', 'impact time (ms)',
       'impact force / body weight', 'adhesive force (mN)',
       'time frog pulls on target (ms)', 'adhesive force / body weight',
       'adhesive impulse (N-s)', 'total contact area (mm2)',
       'contact area without mucus (mm2)',
       'contact area with mucus / contact area without mucus',
       'contact pressure (Pa)', 'adhesive strength (Pa)', 'age', 'SVL (mm)',
       'weight (g)', 'species'],
      dtype='object')

In [16]:
plot = scatter(
    data=df_with_details, 
    x="impact force (mN)", 
    y="adhesive force (mN)", 
    cat="age",
    height=350,
    width=450
)

bokeh.io.show(plot)

Exercise 3.3: Long-term trends in hybridization of Darwin finches

The Grants provided data on the finches of Daphne for the years 1973, 1975, 1987, 1991, and 2012. I have included the data in the files grant_1973.csv, grant_1975.csv, grant_1987.csv, grant_1991.csv, and grant_2012.csv. They are in almost exactly the same format is in the Dryad repository; I have only deleted blank entries at the end of the files.

a) Load each of the files into separate Pandas DataFrames. You might want to inspect the file first to make sure you know what character the comments start with and if there is a header row.

Comment from Emily - I studied the solution to this exercise closely for a long time, so this will be conceptually similar to what Justin does, including using modules you may not know about yet.

In [17]:
import glob

In [18]:
glob.glob?

Signature: glob.glob(pathname, *, recursive=False)
Docstring:
Return a list of paths matching a pathname pattern.

The pattern may contain simple shell-style wildcards a la
fnmatch. However, unlike fnmatch, filenames starting with a
dot are special cases that are not matched by '*' and '?'
patterns.

If recursive is true, the pattern '**' will match any files and
zero or more directories and subdirectories.
File:      c:\users\emily\anaconda3\lib\glob.py
Type:      function


In [19]:
glob.glob("data/grant_*.csv")

['data\\grant_1973.csv',
 'data\\grant_1975.csv',
 'data\\grant_1987.csv',
 'data\\grant_1991.csv',
 'data\\grant_2012.csv',
 'data\\grant_complete.csv',
 'data\\grant_heredity.csv']

In [20]:
#peek at these using command line utilities to make sure we know the structure, comment markers, etc

In [21]:
csv_list = glob.glob("data/grant_*.csv")

csv_list.sort()

# here's a strategy using list comprehensions
file_names = [filepath[5:] for filepath in csv_list[:5]]
df_list = [pd.read_csv(filepath, comment="#") for filepath in csv_list[:5]]

file_names

['grant_1973.csv',
 'grant_1975.csv',
 'grant_1987.csv',
 'grant_1991.csv',
 'grant_2012.csv']

In [22]:
# this is the same thing with a for loop
df_list = []
file_names = []

for filepath in csv_list[:5]:
    file_names.append(filepath[5:])
    df_list.append(pd.read_csv(filepath, comment="#"))

file_names

['grant_1973.csv',
 'grant_1975.csv',
 'grant_1987.csv',
 'grant_1991.csv',
 'grant_2012.csv']

In [23]:
# an alternative lookup strategy
finch_files = {
    filepath[5:-4]: pd.read_csv(filepath, comment="#") for filepath in csv_list[:-2]
}

print(finch_files.keys())
finch_files["grant_1973"].head()

dict_keys(['grant_1973', 'grant_1975', 'grant_1987', 'grant_1991', 'grant_2012'])


,band,species,yearband,beak length,beak depth
0,20123,fortis,73,9.25,8.05
1,20126,fortis,73,11.35,10.45
2,20128,fortis,73,10.15,9.55
3,20129,fortis,73,9.95,8.75
4,20133,fortis,73,11.55,10.15


In [24]:
for df in finch_files.values():
    print(df.columns)

Index(['band', 'species', 'yearband', 'beak length', 'beak depth'], dtype='object')
Index(['band', 'species', 'Beak length, mm', 'Beak depth, mm'], dtype='object')
Index(['band', 'species', 'Beak length, mm', 'Beak depth, mm'], dtype='object')
Index(['band', 'species', 'blength', 'bdepth'], dtype='object')
Index(['band', 'species', 'blength', 'bdepth'], dtype='object')


In [25]:
for df in df_list:
    print(df.columns)
    
df_list[0].head()

Index(['band', 'species', 'yearband', 'beak length', 'beak depth'], dtype='object')
Index(['band', 'species', 'Beak length, mm', 'Beak depth, mm'], dtype='object')
Index(['band', 'species', 'Beak length, mm', 'Beak depth, mm'], dtype='object')
Index(['band', 'species', 'blength', 'bdepth'], dtype='object')
Index(['band', 'species', 'blength', 'bdepth'], dtype='object')


,band,species,yearband,beak length,beak depth
0,20123,fortis,73,9.25,8.05
1,20126,fortis,73,11.35,10.45
2,20128,fortis,73,10.15,9.55
3,20129,fortis,73,9.95,8.75
4,20133,fortis,73,11.55,10.15


b) We would like to merge these all into one DataFrame. The problem is that they have different header names, and only the 1973 file has a year entry (called yearband). This is common with real data. It is often a bit messy and requires some wrangling.

First, change the name of the yearband column of the 1973 data to year. Also, make sure the year format is four digits, not two!
Next, add a year column to the other four DataFrames. You want tidy data, so each row in the DataFrame should have an entry for the year.

In [26]:
years = [int(name[6:10]) for name in file_names]
years

[1973, 1975, 1987, 1991, 2012]

In [27]:
# instead I'm going to add the Year column all at once and then delete the yearband column
for i, _ in enumerate(df_list):
    df_list[i]["year"] = years[i]

df_list[0].head()

,band,species,yearband,beak length,beak depth,year
0,20123,fortis,73,9.25,8.05,1973
1,20126,fortis,73,11.35,10.45,1973
2,20128,fortis,73,10.15,9.55,1973
3,20129,fortis,73,9.95,8.75,1973
4,20133,fortis,73,11.55,10.15,1973


In [28]:
df_list[0].drop(columns="yearband", inplace=True)

df_list[0].head()

,band,species,beak length,beak depth,year
0,20123,fortis,9.25,8.05,1973
1,20126,fortis,11.35,10.45,1973
2,20128,fortis,10.15,9.55,1973
3,20129,fortis,9.95,8.75,1973
4,20133,fortis,11.55,10.15,1973


Change the column names so that all the DataFrames have the same column names. I would choose column names

['band', 'species', 'beak length (mm)', 'beak depth (mm)', 'year']

In [29]:
for df in df_list:
    print(df.columns)

Index(['band', 'species', 'beak length', 'beak depth', 'year'], dtype='object')
Index(['band', 'species', 'Beak length, mm', 'Beak depth, mm', 'year'], dtype='object')
Index(['band', 'species', 'Beak length, mm', 'Beak depth, mm', 'year'], dtype='object')
Index(['band', 'species', 'blength', 'bdepth', 'year'], dtype='object')
Index(['band', 'species', 'blength', 'bdepth', 'year'], dtype='object')


In [30]:
#set all the different column names to point at identical ones
#see Justin's solution for a smoother way to do this

rename_dict = {
    "beak length": "beak length (mm)",
    "Beak length, mm": "beak length (mm)",
    "blength": "beak length (mm)",
    
    "beak depth": "beak depth (mm)",
    "Beak depth, mm": "beak depth (mm)",
    "bdepth": "beak depth (mm)",
}

for i, df in enumerate(df_list):
    df_list[i].rename(columns=rename_dict, inplace=True)

for df in df_list:
    print(df.columns)

Index(['band', 'species', 'beak length (mm)', 'beak depth (mm)', 'year'], dtype='object')
Index(['band', 'species', 'beak length (mm)', 'beak depth (mm)', 'year'], dtype='object')
Index(['band', 'species', 'beak length (mm)', 'beak depth (mm)', 'year'], dtype='object')
Index(['band', 'species', 'beak length (mm)', 'beak depth (mm)', 'year'], dtype='object')
Index(['band', 'species', 'beak length (mm)', 'beak depth (mm)', 'year'], dtype='object')


Concatenate the DataFrames into a single DataFrame. Be careful with indices! If you use pd.concat(), you will need to use the ignore_index=True kwarg. You might also need to use the axis kwarg.

In [31]:
#axis=0 is used by default
df_total = pd.concat(df_list, ignore_index=True)

df_total

,band,species,beak length (mm),beak depth (mm),year
0,20123,fortis,9.25,8.05,1973
1,20126,fortis,11.35,10.45,1973
2,20128,fortis,10.15,9.55,1973
3,20129,fortis,9.95,8.75,1973
4,20133,fortis,11.55,10.15,1973
...,...,...,...,...,...
2299,21295,scandens,14.20,9.30,2012
2300,21297,scandens,13.00,9.80,2012
2301,21340,scandens,14.60,8.90,2012
2302,21342,scandens,13.10,9.80,2012


c) The band field gives the number of the band on the bird's leg that was used to tag it. Are some birds counted twice? Are they counted twice in the same year? Do you think you should drop duplicate birds from the same year? How about different years? My opinion is that you should drop duplicate birds from the same year and keep the others, but I would be open to discussion on that. To practice your Pandas skills, though, let's delete only duplicate birds from the same year from the DataFrame. 

Hint: The DataFrame methods duplicated() and drop_duplicates() will be useful.

In [32]:
df_total.duplicated(subset=["year", "band"])

0       False
1       False
2       False
3       False
4       False
        ...  
2299    False
2300    False
2301    False
2302    False
2303    False
Length: 2304, dtype: bool

In [33]:
sum(df_total.duplicated(subset=["year", "band"]))

5

In [34]:
df_total.drop_duplicates(subset=["year", "band"], inplace=True, ignore_index=True)

# notice the df is 5 fewer rows long now
df_total

,band,species,beak length (mm),beak depth (mm),year
0,20123,fortis,9.25,8.05,1973
1,20126,fortis,11.35,10.45,1973
2,20128,fortis,10.15,9.55,1973
3,20129,fortis,9.95,8.75,1973
4,20133,fortis,11.55,10.15,1973
...,...,...,...,...,...
2294,21295,scandens,14.20,9.30,2012
2295,21297,scandens,13.00,9.80,2012
2296,21340,scandens,14.60,8.90,2012
2297,21342,scandens,13.10,9.80,2012


After doing this work, it is worth saving your tidy DataFrame in a CSV document. To this using the to_csv() method of your DataFrame. Since the indices are uninformative, you should use the index=False kwarg. (I have already done this and saved it as ~/git/bootcamp/data/grant_complete.csv, which will help you do the rest of the exercise if you have problems with this part.)

In [35]:
df_total.to_csv?

Signature:
df_total.to_csv(
    path_or_buf: Union[str, pathlib.Path, IO[~AnyStr], NoneType] = None,
    sep: str = ',',
    na_rep: str = '',
    float_format: Union[str, NoneType] = None,
    columns: Union[Sequence[Union[Hashable, NoneType]], NoneType] = None,
    header: Union[bool, List[str]] = True,
    index: bool = True,
    index_label: Union[bool, str, Sequence[Union[Hashable, NoneType]], NoneType] = None,
    mode: str = 'w',
    encoding: Union[str, NoneType] = None,
    compression: Union[str, Mapping[str, str], NoneType] = 'infer',
    quoting: Union[int, NoneType] = None,
    quotechar: str = '"',
    line_terminator: Union[str, NoneType] = None,
    chunksize: Union[int, NoneType] = None,
    date_format: Union[str, NoneType] = None,
    doublequote: bool = True,
    escapechar: Union[str, NoneType] = None,
    decimal: Union[str, NoneType] = '.',
) -> Union[str, NoneType]
Docstring:
Write object to a comma-separated values (csv) file.

.. versionchanged:: 0.24.0
    Th

In [36]:
df_total.to_csv('tidy_finch_data.csv', index=False)

d) Make a plots exploring how beak depth changes over time for each species. Think about what might be effective ways to display the data.

In [37]:
import bokeh_catplot

In [38]:
bokeh_catplot.box?

Signature:
bokeh_catplot.box(
    data=None,
    cats=None,
    val=None,
    palette=None,
    order=None,
    p=None,
    horizontal=False,
    val_axis_type=None,
    box_width=0.4,
    whisker_caps=False,
    display_points=True,
    outlier_marker='circle',
    min_data=5,
    box_kwargs=None,
    median_kwargs=None,
    whisker_kwargs=None,
    outlier_kwargs=None,
    display_outliers=None,
    **kwargs,
)
Docstring:
Make a box-and-whisker plot from a tidy DataFrame.

Parameters
----------
data : Pandas DataFrame
    DataFrame containing tidy data for plotting.
cats : hashable or list of hastables
    Name of column(s) to use as categorical variable(s).
val : hashable
    Name of column to use as value variable.
palette : list of strings of hex colors, or single hex string
    If a list, color palette to use. If a single string representing
    a hex color, all glyphs are colored with that color. Default is
    colorcet.b_glasbey_category10 from the colorcet package.
order : lis

In [39]:
df_total["species"].unique()

array(['fortis', 'scandens'], dtype=object)

In [40]:
#I wrote a comprehension for this but decided it was less readable
plots = []

for species in df_total["species"].unique():
    plots.append(
        bokeh_catplot.box(
            data=df_total.loc[df_total["species"] == species, :],
            cats=["year"],
            val="beak depth (mm)",
            title=species,
        )
    )

for plot in plots:
    bokeh.io.show(plot)

In [41]:
plots = []

for year in df_total["year"].unique():
    plots.append(
        bokeh_catplot.box(
            data=df_total.loc[df_total["year"] == year, :],
            cats=["species"],
            val="beak depth (mm)",
            title=str(year),
        )
    )

for plot in plots:
    bokeh.io.show(plot)

In [42]:
bokeh.io.show(
    bokeh_catplot.box(
        data=df_total,
        cats=["year", "species"],  # order matters!
        val="beak depth (mm)",
        horizontal=True,
        height=450
    )
)

bokeh.io.show(
    bokeh_catplot.box(
        data=df_total,
        cats=["species", "year"],  # order matters!
        val="beak depth (mm)",
        horizontal=True,
        height=450
    )
)

In [43]:
#Justin's jitter plots
p = bokeh_catplot.strip(
    data=df_total,
    cats=['year', 'species'],
    val='beak depth (mm)',
    color_column='species',
    horizontal=True,
    jitter=True,
    marker_kwargs=dict(alpha=0.2),    #makes them semi transparent
    height=450,
)

bokeh.io.show(p)

e) It is informative to plot the measurement of each bird's beak as a point in the beak depth-beak length plane. For the 1987 data, plot beak depth vs. beak width for Geospiza fortis and for Geospiza scandens. The function you wrote in Exercise 3.1 will be useful to do this.

In [44]:
bokeh.io.show(
    scatter(
        data=df_total.loc[df_total["year"] == 1987, :],
        x="beak length (mm)",
        y="beak depth (mm)",
        cat="species",
        height=400,
        width=600,
        marker_kwargs=dict(alpha=0.4),  # makes dots semi transparent
    )
)

f) Do part (d) again for all years. Hint: To display all of the plots, check out the Bokeh documentation for layouts. In your plots, make sure all plots have the same range on the axes. If you want to set two plots, say p1 and p2 to have the same axis ranges, you can do the following.

p1.x_range = p2.x_range
p1.y_range = p2.y_range

In [45]:
from bokeh.layouts import column

In [46]:
plots = []

for year in df_total["year"].unique():
    plots.append(
        scatter(
            data=df_total.loc[df_total["year"] == year, :],
            x="beak length (mm)",
            y="beak depth (mm)",
            cat="species",
            height=200,
            width=300,
            show_legend=False,
            title=str(year),
            marker_kwargs=dict(alpha=0.4)
        )
    )

bokeh.io.show(column(plots))

In [47]:
#checked above for the biggest ranges

for p in plots:
    p.x_range = plots[2].x_range
    p.y_range = plots[1].y_range
    
bokeh.io.show(column(plots))

In [48]:
#Justin's superior strategy for iterating through the year groups
for name, group in df_total.groupby('year'):
    print('name = ', name)
    print('group = \n', group.head())

name =  1973
group = 
     band species  beak length (mm)  beak depth (mm)  year
0  20123  fortis              9.25             8.05  1973
1  20126  fortis             11.35            10.45  1973
2  20128  fortis             10.15             9.55  1973
3  20129  fortis              9.95             8.75  1973
4  20133  fortis             11.55            10.15  1973
name =  1975
group = 
     band species  beak length (mm)  beak depth (mm)  year
89     2  fortis               9.4              8.0  1975
90     9  fortis               9.2              8.3  1975
91    12  fortis               9.5              7.5  1975
92    15  fortis               9.5              8.0  1975
93   305  fortis              11.5              9.9  1975
name =  1987
group = 
       band species  beak length (mm)  beak depth (mm)  year
488  14613  fortis              9.10             7.00  1987
489  15487  fortis              9.14             7.12  1987
490  15187  fortis              9.24             7.21  